In [13]:
!pip install pulp

# Atividade: Branch & Bound manual em um problema de mochila binária

Nesta tarefa, vamos aplicar Branch & Bound (B&B) manualmente em um problema de mochila binária multidimensional.

 **ATENÇÃO: Não é para programar um B&B é para usar o código para adicionar as restrições de Branching e resolver a relaxação linear.**

## Contexto do problema

Considere o seguinte problema de mochila:
* Tipo: mochila 0–1 (binária)
* Número de itens: 15
* Número de restrições (capacidades): 10

## O modelo matemático é:

\begin{aligned}
\max \quad & \sum_{i=1}^{15} p_i x_i \\
\text{sujeito a} \quad
& \sum_{i=1}^{15} a_{j i} x_i \le b_j, \quad j = 1,\dots,10 \\
& x_i \in \{0,1\}, \quad i = 1,\dots,15
\end{aligned}

## Na implementação em Python, temos:
* $profits[i] = p_i$
* $weights[j][i] = a_{j i}$
* $capacities[j] = b_j$
* $x_i$ são variáveis binárias no modelo inteiro (mas, na relaxação linear, $0 \le x_i \le 1$).

⸻

## Ideia geral da atividade

Vamos construir a árvore de Branch & Bound à mão, usando como nó raiz a relaxação linear do problema (isto é, o modelo de programação linear em que as variáveis x_i são contínuas, $0 \le x_i \le 1$, em vez de binárias).
* A solução inicial vem da solução da relaxação linear (LP) do problema, sem forçar integralidade.
* A partir desta solução, você irá ramificar em variáveis fracionárias, criar nós filhos com restrições adicionais (por exemplo, $x_k = 0$ e $x_k = 1$) e refazer o LP em cada nó.


## Construção da árvore de B&B

1. Nó raiz (relaxação linear)

Resolver o problema de mochila sem integrality, isto é, com:
* $x_i$ como variáveis contínuas entre 0 e 1 ($0 <= x_i <= 1$).

In [14]:
# mdkp_model.py
# Modelo de Programação Inteira Binária para o
# problema da Mochila Multidimensional (15 itens, 10 restrições)

import pulp

# -----------------------------
# 1. Dados do problema
# -----------------------------
N_ITEMS = 15
M_CONS = 10

# Lucros p_i
profits = [
    50, 17, 11, 57, 27,
    25, 24, 18, 57, 16,
    53, 57, 44, 15, 47
]

# Pesos a_{j,i}: lista de listas (cada linha = restrição j)
weights = [
    # j = 0
    [14,  2,  1,  3,  7,  8, 17, 20,  1, 18,  7, 18, 14,  8, 15],
    # j = 1
    [19,  9,  1,  6, 14, 11,  9,  5,  7, 11,  4,  3, 13,  4, 12],
    # j = 2
    [12, 20,  9,  2, 15, 18,  4, 13,  3, 18, 10, 20, 12, 19,  7],
    # j = 3
    [ 3,  2,  8, 10,  3,  8,  4, 13,  9, 15, 12,  6, 12, 12,  7],
    # j = 4
    [ 9,  3, 20,  6, 18,  8,  6, 15, 13,  9, 18,  8, 11,  2,  8],
    # j = 5
    [ 2, 11, 13,  9,  3,  7, 19, 11,  7, 16, 13, 15,  5,  9,  5],
    # j = 6
    [ 8, 18, 18,  9, 19, 14, 19, 13, 12,  8,  5, 17, 16,  3,  2],
    # j = 7
    [ 4,  5,  6, 14, 20,  3, 13, 13, 20, 15, 17,  9, 18,  1,  4],
    # j = 8
    [18,  9, 11,  4, 10, 14,  6, 15,  1,  9, 17,  6, 17,  4, 10],
    # j = 9
    [17, 20,  7,  5, 12,  6, 18, 17,  1, 20, 11, 16,  1,  4, 12],
]

# Capacidades b_j
capacities = [
    64, 60, 48, 60, 70,
    50, 76, 49, 79, 53
]

# -----------------------------
# 2. Criação do modelo MIP
# -----------------------------
def build_mdkp_model():
    # Modelo de maximização
    model = pulp.LpProblem("MDKP_15x10", pulp.LpMaximize)

    # Variáveis binárias x_i
    x = [
        pulp.LpVariable(f"x_{i}", lowBound=0, upBound=1, cat='Continuous',)
        for i in range(N_ITEMS)
    ]

    # Função objetivo: max sum_i p_i x_i
    model += pulp.lpSum(profits[i] * x[i] for i in range(N_ITEMS)), "Objetivo"

    # Restrições de capacidade: sum_i a_{j,i} x_i <= b_j
    for j in range(M_CONS):
        model += (
            pulp.lpSum(weights[j][i] * x[i] for i in range(N_ITEMS))
            <= capacities[j],
            f"cap_{j}"
        )

    return model, x

# -----------------------------
# 3. Resolução Linear
# -----------------------------
if __name__ == "__main__":
    model, x = build_mdkp_model()

    # Resolver com CBC (padrão do PuLP)
    solver = pulp.PULP_CBC_CMD(msg=True)  # msg=True mostra o log
    model.solve(solver)

    # Status da solução
    print("Status:", pulp.LpStatus[model.status])

    # Valor ótimo
    z_opt = pulp.value(model.objective)
    print("Valor ótimo da função objetivo:", z_opt)

    # Valores das variáveis
    print("\nSolução x_i:")
    for i in range(N_ITEMS):
        print(f"x_{i} = {pulp.value(x[i]):.4f}")

    # Checagem das restrições
    print("\nVerificando restrições:")
    for j in range(M_CONS):
        lhs = sum(weights[j][i] * pulp.value(x[i]) for i in range(N_ITEMS))
        print(f"Restrição {j}: LHS = {lhs:.4f}, capacidade = {capacities[j]}")

Status: Optimal
Valor ótimo da função objetivo: 266.33048427

Solução x_i:
x_0 = 1.0000
x_1 = 0.0000
x_2 = 0.0000
x_3 = 1.0000
x_4 = 0.0000
x_5 = 0.2450
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.8632
x_9 = 0.0000
x_10 = 0.0000
x_11 = 1.0000
x_12 = 0.0000
x_13 = 0.0000
x_14 = 1.0000

Verificando restrições:
Restrição 0: LHS = 52.8234, capacidade = 64
Restrição 1: LHS = 48.7379, capacidade = 60
Restrição 2: LHS = 48.0000, capacidade = 48
Restrição 3: LHS = 35.7293, capacidade = 60
Restrição 4: LHS = 44.1823, capacidade = 70
Restrição 5: LHS = 38.7578, capacidade = 50
Restrição 6: LHS = 49.7892, capacidade = 76
Restrição 7: LHS = 49.0000, capacidade = 49
Restrição 8: LHS = 42.2934, capacidade = 79
Restrição 9: LHS = 52.3333, capacidade = 53


2. Primeira ramificação (exemplo já feito)

Em aula e no código de exemplo, já foi mostrada a abertura da árvore em dois ramos:
* Nó 1: $x_0 = 0$
* Nó 2: $x_0 = 1$

Ou seja, a partir do modelo base (relaxação linear), você criou dois modelos:
* model0 = model.copy() + restrição x[0] == 0
* model1 = model.copy() + restrição x[0] == 1

A atividade começa a partir daqui: usando esse exemplo, você deve continuar abrindo a árvore.

## Branch X_0 = 0


In [15]:
# Faz uma cópia do modelo original já construído e resolvido anteriormente.
# Isso é importante porque vamos alterar uma restrição (fixar x[0] = 0)
# sem modificar o modelo original.
model0 = model.copy()

# Adiciona ao novo modelo uma restrição extra:
#     x_0 = 0
# Ou seja, força a variável x[0] a assumir o valor 0.
# Isso equivale a "bloquear" o item 0 e procurar a melhor solução sem ele.
model0 += pulp.LpConstraint(x[0] == 0)

# Resolve o modelo modificado usando o mesmo solver
model0.solve(solver)

# Imprime o status da solução (Optimal, Infeasible, Unbounded etc.)
print("Status:", pulp.LpStatus[model0.status])

# Extrai o valor ótimo da função objetivo do modelo modificado
z_opt = pulp.value(model0.objective)
print("Valor ótimo da função objetivo:", z_opt)

# Imprime os valores das variáveis da solução encontrada
# Note que estamos usando as variáveis x[i] originais,
# que também fazem parte de model0 (pois foram copiadas).
print("\nSolução x_i:")
for i in range(N_ITEMS):
    print(f"x_{i} = {pulp.value(x[i]):.4f}")

Status: Optimal
Valor ótimo da função objetivo: 238.41595454

Solução x_i:
x_0 = 0.0000
x_1 = 0.0000
x_2 = 0.0000
x_3 = 1.0000
x_4 = 0.0000
x_5 = 0.8946
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.9658
x_9 = 0.0000
x_10 = 0.0000
x_11 = 1.0000
x_12 = 0.0000
x_13 = 0.0000
x_14 = 1.0000


## Branch X_0 = 1

In [16]:
# Faz uma cópia do modelo original já construído e resolvido anteriormente.
# Isso é importante porque vamos alterar uma restrição (fixar x[0] = 0)
# sem modificar o modelo original.
model1 = model.copy()

# Adiciona ao novo modelo uma restrição extra:
#     x_0 = 1
# Ou seja, força a variável x[0] a assumir o valor 1.
# Isso equivale a "bloquear" o item 0 e procurar a melhor solução sem ele.
model1 += pulp.LpConstraint(x[0] == 1)

# Resolve o modelo modificado usando o mesmo solver
model1.solve(solver)

# Status da solução
print("Status:", pulp.LpStatus[model1.status])

# Valor ótimo
z_opt = pulp.value(model1.objective)
print("Valor ótimo da função objetivo:", z_opt)

# Valores das variáveis
print("\nSolução x_i:")
for i in range(N_ITEMS):
    print(f"x_{i} = {pulp.value(x[i]):.4f}")

Status: Optimal
Valor ótimo da função objetivo: 266.33048427

Solução x_i:
x_0 = 1.0000
x_1 = 0.0000
x_2 = 0.0000
x_3 = 1.0000
x_4 = 0.0000
x_5 = 0.2450
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.8632
x_9 = 0.0000
x_10 = 0.0000
x_11 = 1.0000
x_12 = 0.0000
x_13 = 0.0000
x_14 = 1.0000


## O que fazer em cada nó da árvore

Para cada nó que você criar na árvore de B&B, faça o seguinte:
1. Escreva as novas restrições que definem aquele nó, por exemplo:
* Nó 3: $x_0 = 0, x_3 = 1$
* Nó 4: $x_0 = 0, x_3 = 0, x_7 = 1$

etc.

2.	Resolva a relaxação linear daquele nó (mesmo modelo, com as capacidades originais, + as restrições de fixação de variáveis que surgem da árvore: x_i=0 ou x_i=1).

3.	Registre, para o nó:
* UB (Upper Bound): valor da solução do LP naquele nó.
* Solução LP: valores de x_i (pelo menos as variáveis importantes/fracionárias).

### Verifique:
* Se o problema ficou inviável → o nó é podado por inviabilidade.
* Se a solução é inteira (todos $x_i \in \{0,1\}$) → você encontrou uma solução viável inteira.
* Atualize o LB global (melhor valor inteiro encontrado até o momento).
* Esse nó é folha (não precisa mais ramificar).

#### Se a solução é fracionária e o UB do nó é:
* menor ou igual ao LB global → pode podar o nó por bound (ele não pode melhorar a melhor solução inteira já conhecida).
* maior que o LB global → o nó ainda é promissor → escolha uma variável fracionária e ramifique, criando dois novos nós:
	•	x_k = 0
	•	x_k = 1

⸻
## Entrega

### Tamanho mínimo da árvore

Você deve construir uma árvore de Branch & Bound com pelo menos 25 nós (incluindo raiz, nós internos e folhas).

Cada nó conta como:
* Um conjunto específico de restrições de fixação (por ex.: x_0=1, x_3=0, x_5=1),
* Mais a solução da relaxação linear correspondente.

Sugestão prática:

* Numere os nós: Nó 0 (raiz), Nó 1, Nó 2, …, Nó 24, etc.

⸻

### Forma de entrega da tarefa

Você pode representar a árvore de B&B de uma das seguintes formas:

	1.	Desenho à mão:
	2.	Planilha (Excel / Google Sheets) conforme exemplo visto em aula:

⸻

### O que entregar

Você deve entregar dois componentes:

	1.	Árvore de Branch & Bound
	2.	Código Python usado para resolver os LPs dos nós


In [17]:
import pulp
import pandas as pd

# ---------------------------------------------------------
# 1. DADOS DO PROBLEMA (Copiados do Notebook)
# ---------------------------------------------------------
N_ITEMS = 15
M_CONS = 10

profits = [
    50, 17, 11, 57, 27, 25, 24, 18, 57, 16,
    53, 57, 44, 15, 47
]

weights = [
    [14,  2,  1,  3,  7,  8, 17, 20,  1, 18,  7, 18, 14,  8, 15],
    [19,  9,  1,  6, 14, 11,  9,  5,  7, 11,  4,  3, 13,  4, 12],
    [12, 20,  9,  2, 15, 18,  4, 13,  3, 18, 10, 20, 12, 19,  7],
    [ 3,  2,  8, 10,  3,  8,  4, 13,  9, 15, 12,  6, 12, 12,  7],
    [ 9,  3, 20,  6, 18,  8,  6, 15, 13,  9, 18,  8, 11,  2,  8],
    [ 2, 11, 13,  9,  3,  7, 19, 11,  7, 16, 13, 15,  5,  9,  5],
    [ 8, 18, 18,  9, 19, 14, 19, 13, 12,  8,  5, 17, 16,  3,  2],
    [ 4,  5,  6, 14, 20,  3, 13, 13, 20, 15, 17,  9, 18,  1,  4],
    [18,  9, 11,  4, 10, 14,  6, 15,  1,  9, 17,  6, 17,  4, 10],
    [17, 20,  7,  5, 12,  6, 18, 17,  1, 20, 11, 16,  1,  4, 12],
]

capacities = [64, 60, 48, 60, 70, 50, 76, 49, 79, 53]

# ---------------------------------------------------------
# 2. FUNÇÃO PARA RESOLVER UM NÓ (RELAXAÇÃO LINEAR)
# ---------------------------------------------------------
def solve_node(fixed_vars):
    """
    Cria e resolve o LP com restrições fixas.
    fixed_vars: dict {indice_item: valor (0 ou 1)}
    Retorna: (status, valor_objetivo, solucao_dict)
    """
    # Define o problema como Maximização
    model = pulp.LpProblem("MDKP_Node", pulp.LpMaximize)

    # Variáveis Contínuas (Relaxação Linear 0 <= x <= 1)
    x = [pulp.LpVariable(f"x_{i}", 0, 1, pulp.LpContinuous) for i in range(N_ITEMS)]

    # Função Objetivo
    model += pulp.lpSum(profits[i] * x[i] for i in range(N_ITEMS))

    # Restrições de Capacidade
    for j in range(M_CONS):
        model += pulp.lpSum(weights[j][i] * x[i] for i in range(N_ITEMS)) <= capacities[j]

    # Restrições de Branching (Fixação de variáveis deste nó)
    for idx, val in fixed_vars.items():
        model += (x[idx] == val)

    # Resolve (silencioso)
    solver = pulp.PULP_CBC_CMD(msg=False)
    model.solve(solver)

    status = pulp.LpStatus[model.status]
    obj_val = pulp.value(model.objective) if status == "Optimal" else -1.0

    sol_vals = {}
    if status == "Optimal":
        for i in range(N_ITEMS):
            sol_vals[i] = pulp.value(x[i])

    return status, obj_val, sol_vals

# ---------------------------------------------------------
# 3. ALGORITMO BRANCH & BOUND (Gerador da Árvore)
# ---------------------------------------------------------

# Lista para armazenar o histórico dos nós (para seu relatório)
tree_history = []

# Fila de nós para explorar (Estratégia: Depth First Search - Pilha)
# Estrutura: {'id': int, 'parent': int, 'fixed': dict, 'desc': str}
stack = [{'id': 0, 'parent': -1, 'fixed': {}, 'desc': 'Raiz'}]

best_lb = -1  # Melhor solução inteira encontrada (Lower Bound)
best_sol_int = {}
node_counter = 0

MAX_NODES = 80 # Limite de segurança para o exercício (pede 25)

while stack and node_counter < MAX_NODES:
    # Pega o próximo nó
    current_node = stack.pop(0) # Use pop(0) para Breadth-First (BFS) ou pop() para Depth-First (DFS)
    # BFS (pop(0)) é melhor para desenhar árvores didáticas equilibradas nivel a nivel

    node_id = current_node['id']
    fixed = current_node['fixed']

    # 1. Resolve a Relaxação
    status, ub, sol = solve_node(fixed)

    # Prepara dados para o relatório
    node_info = {
        "Nó": node_id,
        "Pai": current_node['parent'],
        "Restrição Adicionada": current_node['desc'],
        "Status": status,
        "UB (Relaxação)": round(ub, 4) if ub else 0,
        "Ação": "",
        "Variável Ramificada": "-"
    }

    # 2. Análise do Nó
    if status != "Optimal":
        node_info["Ação"] = "PODA (Infactível)"
        tree_history.append(node_info)
        continue

    # Verifica integrality (se todas as variáveis são 0 ou 1)
    is_integer = True
    first_fractional = -1

    # Procura a primeira variável fracionária para ramificar
    # (Estratégia: menor índice fracionário)
    for i in range(N_ITEMS):
        val = sol[i]
        if not (abs(val - 0) < 1e-5 or abs(val - 1) < 1e-5):
            is_integer = False
            if first_fractional == -1:
                first_fractional = i

    # PODA POR BOUND: Se o UB deste nó for pior ou igual ao melhor inteiro já achado
    # (Nota: como é float, usamos uma tolerância pequena)
    if ub <= best_lb + 1e-5:
        node_info["Ação"] = "PODA (Bound)"
        tree_history.append(node_info)
        continue

    # Se achou solução inteira
    if is_integer:
        node_info["Ação"] = "PODA (Solução Inteira)"
        if ub > best_lb:
            best_lb = ub
            best_sol_int = sol
            node_info["Ação"] += " - NOVO INCUMBENTE!"
        tree_history.append(node_info)
        continue

    # Se chegou aqui, precisa RAMIFICAR
    node_info["Ação"] = "Ramificar"
    node_info["Variável Ramificada"] = f"x_{first_fractional}"

    # Cria filhos (Branching)
    # Filho da Esquerda: x_k = 0
    fixed_0 = fixed.copy()
    fixed_0[first_fractional] = 0
    node_counter += 1
    stack.append({
        'id': node_counter,
        'parent': node_id,
        'fixed': fixed_0,
        'desc': f"x_{first_fractional} = 0"
    })

    # Filho da Direita: x_k = 1
    fixed_1 = fixed.copy()
    fixed_1[first_fractional] = 1
    node_counter += 1
    stack.append({
        'id': node_counter,
        'parent': node_id,
        'fixed': fixed_1,
        'desc': f"x_{first_fractional} = 1"
    })

    tree_history.append(node_info)

# ---------------------------------------------------------
# 4. EXIBIÇÃO DOS RESULTADOS
# ---------------------------------------------------------
df = pd.DataFrame(tree_history)
print(f"Melhor Solução Inteira Encontrada: {best_lb}")
print("\n--- TABELA DA ÁRVORE DE B&B (Primeiros nós) ---")
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
# Mostra colunas principais
print(df[['Nó', 'Pai', 'Restrição Adicionada', 'UB (Relaxação)', 'Ação', 'Variável Ramificada']])

# Opcional: Salvar em CSV para abrir no Excel
# df.to_csv("arvore_bnb.csv", index=False)

Melhor Solução Inteira Encontrada: -1

--- TABELA DA ÁRVORE DE B&B (Primeiros nós) ---
    Nó  Pai Restrição Adicionada  UB (Relaxação)       Ação Variável Ramificada
0    0   -1                 Raiz        266.3305  Ramificar                 x_5
1    1    0              x_5 = 0        265.0716  Ramificar                 x_8
2    2    0              x_5 = 1        255.6480  Ramificar                x_10
3    3    1              x_8 = 0        252.1569  Ramificar                x_10
4    4    1              x_8 = 1        262.3409  Ramificar                 x_3
5    5    2             x_10 = 0        254.5625  Ramificar                 x_6
6    6    2             x_10 = 1        245.0088  Ramificar                 x_0
7    7    3             x_10 = 0        243.8057  Ramificar                 x_6
8    8    3             x_10 = 1        249.1461  Ramificar                x_11
9    9    4              x_3 = 0        246.2870  Ramificar                x_10
10  10    4              x_3 = 1 